In [1]:
import libsumo as traci
import time
import csv
import os
import random, numpy as np
import pandas as pd, matplotlib.pyplot as plt
import csv, os
from datetime import datetime
from src import config

random.seed(config.SEED)
np.random.seed(config.SEED)

## Simulation Section

In [2]:
from src.simulation_libs import find_avg_halt_range;
from src import config
from src import road_config

sumoCmd = [
    config.SUMO_BINARY,
    "-c", config.SUMO_CFG,
    "--seed", str(config.SEED),
]
traci.start(sumoCmd)
print(traci.trafficlight.getControlledLanes(road_config.TL_ID))
traci.simulationStep()
traci.simulation.saveState(config.SUMO_FIRST_STATE)
signal_state = traci.trafficlight.getRedYellowGreenState(road_config.TL_ID)
print(signal_state)

(max_avg_halt,min_avg_halt,max_fairness,min_fairness) = find_avg_halt_range(road_config.NS_LANES, road_config.EW_LANES)
print('Max avg halt', max_avg_halt)
print('Min avg halt', min_avg_halt)
print('Max fairness', max_fairness)
print('Min fairness', min_fairness)
traci.close()

('1265822568#3_0', '1265822568#3_0', '1265822568#3_1', '1265822568#3_2', '1265822568#3_2', '1200728225#1_0', '1200728225#1_1', '1200728225#1_1', '1200728225#1_1', '1221994726#0_0', '1221994726#0_0', '1221994726#0_1')
rrrrrGGggGGG
❌ Stopping at END_TIME.
Max avg halt 29.0
Min avg halt 3.0641025641025643
Max fairness 0.9999999824868655
Min fairness 0.00494699643146999


In [ ]:
from src.de_experiment import seed_de_simulation
# Run the function
seed_de_simulation()

In [ ]:
from src import config
from src.utils import visualize_pow_results

visualize_pow_results(log_dir=config.LOG_DIR, suffix="20251108_101717")

In [ ]:
from src.de_experiment import baseline_simulation

baseline_simulation()

In [ ]:
from src import config
from src.utils import visualize_pow_results

visualize_pow_results(log_dir=config.LOG_DIR, prefix="traffic_baseline", suffix="20251108_111006")

In [ ]:
from src.utils import compare_logs_by_time

compare_logs_by_time(de_suffix='20251108_101717', baseline_suffix='20251108_111006')

In [ ]:
from src.utils import compare_score_best

compare_score_best(de_suffix='20251108_101717', baseline_suffix='20251108_111006')